In [3]:
from subprocess import check_output
import datetime
import arrow as ar
import pandas as pd
import numpy as np


# print(check_output(["ls", "./data"]).decode("utf8"))


# Any results you write to the current directory are saved as output.
input_path = "/Users/dongjian/data/meituanKaggleData/"
output_path = "/Users/dongjian/data/meituanKaggleData/"

def init_arguments():
    parser = argparse.ArgumentParser()

    parser.add_argument('--input', type=str, dest='input_path',
                        help="data")
    parser.add_argument('--output', type=str, dest='output_path',
                        help="data")
    parser.add_argument('-k', type=str, dest="key",
                        help="map key(rider or rider_poi)")
    return parser.parse_args()


def load_order_data(file_name, sep=","):
    df = pd.read_csv('%s' % file_name, sep)
    names = map(lambda x: x.replace("algocamp_order_test.", ""), df.columns.tolist())

    df.columns = names
    # df.rename(columns=names,inplace=True)
    c = 'order_unix_time'
    mask = pd.notnull(df[c])
    df.loc[mask, "tmp_order_unix_time"] = df.loc[mask, c]
    df.loc[mask, c] = df.loc[mask, c].apply(lambda x: ar.get(x).to("local"))
    df.loc[mask, 'day'] = df.loc[mask, c].apply(lambda x: x.day)
    df.loc[mask, 'weekday'] = df.loc[mask, c].apply(lambda x: x.weekday())
    df.loc[mask, 'hour'] = df.loc[mask, c].apply(lambda x: x.hour)
    df.loc[mask, '10min'] = df.loc[mask, c].apply(lambda x: x.minute / 10)
    df.loc[mask, 'high'] = df.loc[mask, c].apply(lambda x: 1 if x.hour in (11, 12, 13, 17, 18, 19) else 0)
    df.loc[mask, 'weekday_hour'] = df.loc[mask, c].apply(lambda x: "_".join([str(x.weekday()), str(x.hour)]))
    df.loc[mask, c] = df.loc[mask, "tmp_order_unix_time"]
    return df


def load_area_data(file_name):
    df = pd.read_csv(file_name, dtype={'date': str, 'time': str})
    c = "log_unix_time"
    mask = pd.notnull(df[c])
    df.loc[mask, c] = df.loc[mask, c].apply(lambda x: ar.get(x).to("local"))
    df.loc[mask, 'day'] = df.loc[mask, c].apply(lambda x: x.day)
    df.loc[mask, 'weekday'] = df.loc[mask, c].apply(lambda x: x.weekday())
    df.loc[mask, 'hour'] = df.loc[mask, c].apply(lambda x: x.hour)
    df.loc[mask, '10min'] = df.loc[mask, c].apply(lambda x: x.minute / 10)
    df.loc[mask, 'high'] = df.loc[mask, c].apply(lambda x: 1 if x.hour in (11, 12, 13, 17, 18, 19) else 0)
    df.loc[mask, 'weekday_hour'] = df.loc[mask, c].apply(lambda x: "_".join([str(x.weekday()), str(x.hour)]))
    df = df.loc[df.not_fetched_order_num > 0]
    return df


def load_weather_data(file_name):
    df = pd.read_csv(file_name, dtype={'date': str, 'time': str})
    c = "log_unix_time"
    mask = pd.notnull(df[c])
    df.loc[mask, c] = df.loc[mask, c].apply(lambda x: ar.get(x).to("local"))
    df.loc[mask, 'day'] = df.loc[mask, c].apply(lambda x: x.day)
    df.loc[mask, 'weekday'] = df.loc[mask, c].apply(lambda x: x.weekday())
    df.loc[mask, 'hour'] = df.loc[mask, c].apply(lambda x: x.hour)
    df.loc[mask, 'weekday_hour'] = df.loc[mask, c].apply(lambda x: "_".join([str(x.weekday()), str(x.hour)]))
    df.loc[mask, 'high'] = df.loc[mask, c].apply(lambda x: 1 if x.hour in (11, 12, 13, 17, 18, 19) else 0)
    df = df.loc[df.temperature > -30]
    return df


def load_data(path_data):
    '''
    --------------------------------order_product--------------------------------
    * Unique in order_id + product_id
    '''
    waybill_info = load_order_data(path_data + 'waybill_info.csv')
    area_realtime = load_area_data(path_data + 'area_realtime.csv')
    weather_realtime = load_weather_data(path_data + 'weather_realtime.csv')
    waybill_info_test_a = load_order_data(path_data + 'waybill_info_test_a.csv', sep="\t")
    waybill_info_test_b = load_order_data(path_data + 'waybill_info_test_b.csv')
    area_realtime_test = load_area_data(path_data + 'area_realtime_test.csv')
    weather_realtime_test = load_weather_data(path_data + 'weather_realtime_test.csv')

    return waybill_info, area_realtime, weather_realtime, waybill_info_test_a, waybill_info_test_b, area_realtime_test, weather_realtime_test


def gene_mask(df, h, high=False):
    base = (df.delivery_duration < 4654.0) & (df.delivery_duration > 663.0)
    if high:
        return base & (df.high.values == 1)
    else:
        return base & ((df.hour.values == h)|(df.hour.values == h+6))

def bearing_array(lat1, lng1, lat2, lng2):
    AVG_EARTH_RADIUS = 6371  # in km
    lng_delta_rad = np.radians(lng2 - lng1)
    lat1, lng1, lat2, lng2 = map(np.radians, (lat1, lng1, lat2, lng2))
    y = np.sin(lng_delta_rad) * np.cos(lat2)
    x = np.cos(lat1) * np.sin(lat2) - np.sin(lat1) * np.cos(lat2) * np.cos(lng_delta_rad)
    return np.degrees(np.arctan2(y, x))

In [5]:
waybill_info, area_realtime, weather_realtime, waybill_info_test_a, waybill_info_test_b, area_realtime_test, weather_realtime_test = load_data(
            input_path)

ori = waybill_info.merge(weather_realtime, on=['day', 'hour', 'area_id'], how='left') \
        .merge(area_realtime, on=['day', 'hour', 'area_id','10min'], how='left')

In [ ]:
print len(ori)

In [4]:
#basic

pre = ori.loc[
        gene_mask(ori, 10) & (ori.day != 30) & (ori.day != 29) & (ori.day != 28) & (ori.day != 27) & (ori.day != 26) & (
            ori.day != 25) & (ori.day != 24)]

tr = ori.loc[
        gene_mask(ori,0, True) & gene_mask(12, ori) & gene_mask(13, ori) & (ori.day != 30) & (ori.day != 29) & (
            ori.day != 28) & (ori.day != 27) & (ori.day != 26) & (
            ori.day != 25) & (ori.day != 24)]

ta = ori.loc[
        gene_mask(ori, 10) & (
            (ori.day == 30) | (ori.day == 29) | (ori.day == 28) | (ori.day == 27) | (ori.day == 26) | (
                ori.day == 25) | (
                ori.day == 24))]

te = ori.loc[
        gene_mask(ori,10) & (
            (ori.day == 30) | (ori.day == 29) | (ori.day == 28) | (ori.day == 27) | (ori.day == 26) | (
                ori.day == 25) | (
                ori.day == 24))]
print "pre.size,tr.size,te.size,ta.size ", len(pre), len(tr), len(te), len(ta)

KeyboardInterrupt: 

In [ ]:
#basic feature
tr["arriveshop_cost"] = tr["arriveshop_unix_time"].apply(lambda x: int(x)) - tr["tmp_order_unix_time"].apply(
            lambda x: int(x))
tr["fetch_cost"] = tr["fetch_unix_time"] - tr["arriveshop_unix_time"]
tr["arrive_guest_cost"] = tr["finish_unix_time"] - tr["fetch_unix_time"]

tr.loc[:, "avg_speed"] = tr["delivery_distance"] / tr["delivery_duration"]
tr.loc[:, 'direction'] = bearing_array(tr['poi_lat'].values, tr['poi_lng'].values, tr['customer_latitude'].values, tr['customer_longitude'].values)
tr.loc[:, 'poi_lat_bin'] = np.round(train['poi_lat'], 2)
tr.loc[:, 'poi_lng_bin'] = np.round(train['poi_lng'], 2)
tr.loc[:, 'cst_lat_bin'] = np.round(train['customer_latitude'], 2)
tr.loc[:, 'cst_lng_bin'] = np.round(train['customer_longitude'], 2)

te.loc[:, "avg_speed"] = tr["delivery_distance"] / tr["delivery_duration"]
te.loc[:, 'direction'] = bearing_array(tr['poi_lat'].values, tr['poi_lng'].values, tr['customer_latitude'].values, tr['customer_longitude'].values)
te.loc[:, 'poi_lat_bin'] = np.round(train['poi_lat'], 2)
te.loc[:, 'poi_lng_bin'] = np.round(train['poi_lng'], 2)
te.loc[:, 'cst_lat_bin'] = np.round(train['customer_latitude'], 2)
te.loc[:, 'cst_lng_bin'] = np.round(train['customer_longitude'], 2)

In [ ]:
#cluster
from sklearn.cluster import MiniBatchKMeans
coords = np.vstack((tr[['poi_lat', 'poi_lng']].values,
                    tr[['customer_latitude', 'customer_longitude']].values,
                    te[['poi_lat', 'poi_lng']].values,
                    te[['poi_lat', 'poi_lng']].values))
sample_ind = np.random.permutation(len(coords))[:500000]
kmeans = MiniBatchKMeans(n_clusters=100, batch_size=10000).fit(coords[sample_ind])
train.loc[:, 'pickup_cluster'] = kmeans.predict(train[['pickup_latitude', 'pickup_longitude']])
train.loc[:, 'dropoff_cluster'] = kmeans.predict(train[['dropoff_latitude', 'dropoff_longitude']])
test.loc[:, 'pickup_cluster'] = kmeans.predict(test[['pickup_latitude', 'pickup_longitude']])
test.loc[:, 'dropoff_cluster'] = kmeans.predict(test[['dropoff_latitude', 'dropoff_longitude']])
t1 = dt.datetime.now()
print('Time till clustering: %i seconds' % (t1 - t0).seconds)

In [ ]:
#speed feature
for gby_col in ['hour', 'weekday','10min',
               'weekday_hour', 'pickup_cluster', 'dropoff_cluster']:
    
    
for gby_cols in [['hour', 'pickup_cluster'],  ['hour', 'dropoff_cluster'],
                 ['pickup_cluster', 'dropoff_cluster']]: